In [192]:
%matplotlib inline
import pandas as pd
import numpy as np
import re
import os
from matplotlib.ticker import MaxNLocator
import matplotlib.pyplot as plt
from requests import get
from bs4 import BeautifulSoup
from pandas.util import hash_pandas_object

In [193]:
data = pd.read_csv('../data/en.openfoodfacts.org.products.csv', sep = '\t')

In [194]:
header = data.columns
odf = pd.DataFrame()
for index in header:
    odf[index] =( [index , sum(data[index].notnull())/1023451*100  , data[index].value_counts().head(4).index.tolist() ])
odf=odf.T
odf =odf.drop(columns=0)
odf=odf.rename(columns={1: "%", 2: "exemple top-4"})

In [195]:
odfcut=odf[odf["%"]>=3]

In [196]:
odfcut.to_csv('odfcut.csv')


Categories each product according to the PNNS

In [197]:
pnns_category = {'Dairies','Composite','Fish Meat Eggs','Beverages','Fat Sauces','Fruits Vegetables','Starchy','Snacks'}

In [198]:
data.pnns_groups_1.value_counts()

unknown                    716232
Sugary snacks               57997
Milk and dairy products     43467
Fish Meat Eggs              42475
Cereals and potatoes        34187
Beverages                   29118
Fat and sauces              28114
Composite foods             25430
Fruits and vegetables       24419
Salty snacks                19825
sugary-snacks                3469
fruits-and-vegetables        2898
cereals-and-potatoes           43
salty-snacks                    5
Name: pnns_groups_1, dtype: int64

In [199]:
pnns1_category = {'unknown' : 'Unknown',
 'Sugary snacks' : 'Snacks', 
 'Milk and dairy products' : 'Dairies',
 'Composite foods' : 'Composite', 
 'Cereals and potatoes' : 'Starchy', 
 'Fish Meat Eggs' : 'Fish Meat Eggs',
 'Beverages' : 'Beverages',
 'Fat and sauces' : 'Fat Sauces',
 'Fruits and vegetables' : 'Fruits Vegetables',
 'Salty snacks' : 'Snacks',
 'fruits-and-vegetables' : 'Fruits Vegetables',
 'sugary-snacks' : 'Snacks',
 'cereals-and-potatoes' : 'Starchy',
 'salty-snacks' : 'Snacks'
}
data['Category_Food'] = data['pnns_groups_1'].replace(pnns1_category)

In [200]:
data.Category_Food.value_counts()

Unknown              716232
Snacks                81296
Dairies               43467
Fish Meat Eggs        42475
Starchy               34230
Beverages             29118
Fat Sauces            28114
Fruits Vegetables     27317
Composite             25430
Name: Category_Food, dtype: int64

More category from 'main_category_en' source

In [201]:
data['main_category_en'].value_counts().head(30)

Plant-based foods and beverages    86518
Snacks                             51656
Beverages                          43688
Dairies                            38129
Meats                              31497
Groceries                          23517
Meals                              17557
Desserts                            7435
Spreads                             7272
Seafood                             7198
Fruit juices                        6415
Canned foods                        4731
Frozen foods                        4138
Biscuits and cakes                  3412
Sweeteners                          2488
Dietary supplements                 1995
Breakfasts                          1861
Vinegars                            1764
Sandwiches                          1760
Fats                                1674
Farming products                    1631
Fish and meat and eggs              1604
Fruit-juices-and-nectars            1241
Baby foods                          1231
Pies            

In [202]:
maincategoryen_category = {'unknown' : 'Unknown',
'Plant-based foods and beverages' : 'Brevages',
'Snacks' : 'Snacks',
'Beverages' : 'Beverages',
'Dairies' : 'Dairies' ,
'Meats' : 'Fish Meat Eggs',
'Meals' : 'Composite',
'Desserts' : 'Snacks',
'Seafood' : 'Fish Meat Eggs',
'Fruit juices' : 'Brevages',
'Biscuits and cakes' : 'Snacks',
'Fats' : 'Fat Sauces', 
'Fish and meat and eggs' : 'Fish Meat Eggs',
'Fruit-juices-and-nectars' : 'Brevages'
                          }

data['Category_Food_frommaincategoryen'] = data['main_category_en'].replace(maincategoryen_category)

In [203]:
data['Category_Food_frommaincategoryen'] = np.where(data['Category_Food_frommaincategoryen'].isin(pnns_category)==False,'Unknown',data['Category_Food_frommaincategoryen'])

In [204]:
data['Category_Food']= np.where(data['Category_Food']=='Unknown', data['Category_Food_frommaincategoryen'], data['Category_Food'])
                                                   

In [206]:
data.pnns_groups_1.value_counts()

unknown                    716232
Sugary snacks               57997
Milk and dairy products     43467
Fish Meat Eggs              42475
Cereals and potatoes        34187
Beverages                   29118
Fat and sauces              28114
Composite foods             25430
Fruits and vegetables       24419
Salty snacks                19825
sugary-snacks                3469
fruits-and-vegetables        2898
cereals-and-potatoes           43
salty-snacks                    5
Name: pnns_groups_1, dtype: int64

In [205]:
data.Category_Food.value_counts()

Unknown              700489
Snacks                86055
Dairies               45034
Fish Meat Eggs        42484
Beverages             38526
Starchy               34230
Fat Sauces            28114
Fruits Vegetables     27317
Composite             25430
Name: Category_Food, dtype: int64

In [212]:
data[data['Category_Food']=='Unknown']['categories'].value_counts()

Compléments alimentaires                                                                                                                                                                                                                         350
Protein bars                                                                                                                                                                                                                                     328
Pancake mixes                                                                                                                                                                                                                                    260
Licensed products                                                                                                                                                                                                                                258
Maple syrups        